# Get Whataburger locations

#### Load Python tools and Jupyter config

In [1]:
import us 
import re
import json
import black
import random
import requests
import numpy as np
import pandas as pd
import jupyter_black
import altair as alt
from time import sleep
import geopandas as gpd
from random import randint
from bs4 import BeautifulSoup
from vega_datasets import data
from tqdm.notebook import tqdm, trange

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
place = "whataburger"
place_formal = "Whataburger"
color = "#fe7800"

---

## Scrape

#### Headers for requests

In [4]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"
}

#### Read store list page

In [5]:
url = "https://locations.whataburger.com/directory.html"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

#### Find all the state links in the directory and store them in a list

In [6]:
locations_list = []

for a in tqdm(soup.findAll("a", class_="Directory-listLink")):
    locations_dict = {
        "url": (
            "https://locations.whataburger.com/" + a["href"].split("/", 1)[0] + ".html"
        ).replace(".html.html", ".html"),
        "state": a.text,
    }
    locations_list.append(locations_dict)
locations_df = pd.DataFrame(locations_list)

  0%|          | 0/15 [00:00<?, ?it/s]

#### Loop through every state, snagging links to cities

In [7]:
city_dir_list = []

for s in tqdm(locations_df["url"].to_list()):
    city_response = requests.get(s, headers=headers)
    city_soup = BeautifulSoup(city_response.text, "html.parser")
    city_directory = city_soup.findAll("li", class_="Directory-listItem")
    for t in city_directory:
        city_dir_dict = {
            "url": "https://locations.whataburger.com/" + t.find("a")["href"],
            "city": t.text.title(),
            "count": t.find("span")
            .attrs["data-count"]
            .replace("(", "")
            .replace(")", ""),
        }
        city_dir_list.append(city_dir_dict)

  0%|          | 0/15 [00:00<?, ?it/s]

In [8]:
city_dir_src = pd.DataFrame(city_dir_list)

#### Some cities have multiple restaurants. Others have just one (single) link. 

In [9]:
city_dir_df_plural = city_dir_src.query('count > "1"')
city_dir_df_single = city_dir_src.query('count == "1"')

#### Loop through those with one or more places (plural) and get links to individual restaurants

In [10]:
plural_list = []

for p in tqdm(city_dir_df_plural["url"].to_list()):
    plural_response = requests.get(p, headers=headers)
    plural_soup = BeautifulSoup(plural_response.text, "html.parser")
    plural_directory = plural_soup.findAll("div", class_="Teaser-row Teaser-row--info")
    for pld in plural_directory:
        plural_list.append(
            "https://locations.whataburger.com"
            + pld.find("a", class_="Teaser-titleLink")["href"].replace("..", "")
        )

  0%|          | 0/133 [00:00<?, ?it/s]

#### Create two lists of the single and plural places, and merge them

In [11]:
single_list = city_dir_df_single["url"].to_list()

In [12]:
location_url_list = plural_list + single_list

#### Loop through that list, snagging details about each location

In [13]:
lul_list = []

for lul in tqdm(location_url_list):
    lul_response = requests.get(lul, headers=headers)
    lul_soup = BeautifulSoup(lul_response.text, "html.parser")
    lul_dict = {
        "id": lul_soup.find("h1").text.replace("Welcome to Whataburger ", ""),
        "location": lul_soup.find("span", class_="Banner-titleGeo").text,
        "street": lul_soup.find("span", class_="c-address-street-1").text.title(),
        "city": lul_soup.find("span", class_="c-address-city").text.title(),
        "state": lul_soup.find("span", class_="c-address-state").text.title(),
        "zipcode": lul_soup.find("span", class_="c-address-postal-code").text.title(),
        "phone": lul_soup.find(
            "div", class_="Phone-display Phone-display--withLink"
        ).text.title(),
        "latitude": json.loads(
            lul_soup.findAll("script", class_="js-map-data")[0].text
        )["latitude"],
        "longitude": json.loads(
            lul_soup.findAll("script", class_="js-map-data")[0].text
        )["longitude"],
        "url": lul,
    }
    lul_list.append(lul_dict)

  0%|          | 0/1014 [00:00<?, ?it/s]

#### Make a dataframe with each location

In [14]:
df = pd.DataFrame(lul_list)

In [15]:
df.head()

,id,location,street,city,state,zipcode,phone,latitude,longitude,url
0,#1116,HWY 280 & Resource Center,111 Resource Center Pkwy,Birmingham,Alabama,35242,(205) 980-9315,33.425566,-86.705371,https://locations.whataburger.com/al/birmingham/111-resource-center-pkwy.html
1,# 896,CR 10 & Trussville,5931 Trussville Crossings Pkwy,Birmingham,Alabama,35235,(205) 655-4513,33.642994,-86.625934,https://locations.whataburger.com/al/birmingham/5931-trussville-crossings-pkwy.html
2,#1203,Memorial Pkwy & Hobbs Rd,11701 S Memorial Pkwy,Huntsville,Alabama,35803,(256) 203-8030,34.617277,-86.566448,https://locations.whataburger.com/al/huntsville/11701-s-memorial-pkwy.html
3,#1249,Friendship Commons & Winchester Road NE,2056 Winchester Road Ne,Huntsville,Alabama,35811,(256) 217-4249,34.815141,-86.498209,https://locations.whataburger.com/al/huntsville/2056-winchester-road-ne.html
4,#1236,,5900 University Dr Nw,Huntsville,Alabama,35806,(256) 417-6578,34.742133,-86.665901,https://locations.whataburger.com/al/huntsville/5900-university-dr-nw.html


In [16]:
len(df)

1014

---

## Geography

#### Make it a geodataframe

In [17]:
df_geo = df.copy()

In [18]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

In [19]:
locations_gdf = gdf.set_crs("EPSG:4326").copy()

---

## Maps

#### US states background

In [20]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [21]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=10, color=color)
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [22]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value(color),
        tooltip=["state:N", "count:Q"],
    )
    .properties(
        title=f"Number of {place_formal} in US, by average lon/lat of locations"
    )
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [23]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [24]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [25]:
locations_gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)